# Предсказание увольнения сотрудника

Необходимо создать модели для предсказания удовлетворённостью работой и вероятностью увольнения сотрудников.

## Цель
С помощью модели необходимо предсказывать удовлетворённость и вероятность ухода из компании сотрудников:
 - выбрать алгоритм и обучить модель регрессии, предсказвающую удовлетворённость сотрудника своим местом работы;
 - выбрать алгоритм и обучить модель классификации, предсказывающую увольнение сотрундика;
 - выявить связь удовлетворённости рабочим местом и вероятностью его увольнения
 - построить основную уточнённую модель предсказания вероятности увольнения сотрудника 

## Описание данных:
### Характеристики сотрудника
- `id` — уникальный идентификатор сотрудника;
- `dept` — отдел, в котором работает сотрудник;
- `level` — уровень занимаемой должности;
- `workload` — уровень загруженности сотрудника;
- `employment_years` — длительность работы в компании (в годах);
- `last_year_promo` — показывает, было ли повышение за последний год;
- `last_year_violations` — показывает, нарушал ли сотрудник трудовой договор за последний год;
- `supervisor_evaluation` — оценка качества работы сотрудника, которую дал руководитель;
- `salary` — ежемесячная зарплата сотрудника;
- `job_satisfaction_rate` — уровень удовлетворённости сотрудника работой в компании, целевой признак.
### Увольнение сотрудника
- `id` — уникальный идентификатор сотрудника;
- `quit` — признак увольнения сотрудника.

## План исследования

1) Предобработка данных
2) Исследовательский анализ данных
3) Подготовка данных к ML
4) Корреляционный анализ
5) Построение пайплайнов для выбора лучшей модели регрессии
6) Анализ важности признаков регрессии
7) Построение пайплайнов для выбора лучшей модели классификации
8) Анализ важности признаков классификации
9) Исследование связи удовлетворённости сотрудника работой и вероятностью его увольнения
10) Построение основной уточнённой модели предсказания вероятности увольнения сотрудника
11) Общий вывод

##  Предобработка данных

### Импорт модулей

! pip install shap
! pip install phik

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import shap
import random
import phik
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.cm import ScalarMappable
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, RobustScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import roc_auc_score, confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

Определение констант

In [2]:
RANDOM_STATE = 1

Загрузка данных

In [3]:
try:
    X_test = pd.read_csv(r"C:\Users\yka\Documents\Practicum\DS_Practicum\Composite_Project_2_HR_Analytics\source\test_features.csv")
    y_test_classif = pd.read_csv(r"C:\Users\yka\Documents\Practicum\DS_Practicum\Composite_Project_2_HR_Analytics\source\test_target_quit.csv")
    y_test_regress = pd.read_csv(r"C:\Users\yka\Documents\Practicum\DS_Practicum\Composite_Project_2_HR_Analytics\source\test_target_job_satisfaction_rate.csv")
    X_train = pd.read_csv(r"C:\Users\yka\Documents\Practicum\DS_Practicum\Composite_Project_2_HR_Analytics\source\train_job_satisfaction_rate.csv")
except:
    X_test = pd.read_csv(r"/datasets/test_features.csv")
    y_test_classif = pd.read_csv(r"/datasets/test_target_quit.csv")
    y_test_regress = pd.read_csv(r"/datasets/test_target_job_satisfaction_rate.csv")
    X_train = pd.read_csv(r"/datasets/train_job_satisfaction_rate.csv")

pd.set_option('display.max_columns', None)

display(X_test.head(20))
display(y_test_classif.head(20))
display(y_test_regress.head(20))
display(X_train.head(20))

,id,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary
0,485046,marketing,junior,medium,2,no,no,5,28800
1,686555,hr,junior,medium,1,no,no,4,30000
2,467458,sales,middle,low,5,no,no,4,19200
3,418655,sales,middle,low,6,no,no,4,19200
4,789145,hr,middle,medium,5,no,no,5,40800
5,429973,purchasing,middle,medium,6,no,no,3,42000
6,850699,technology,junior,medium,3,no,no,3,26400
7,500791,sales,middle,high,9,no,no,4,49200
8,767867,marketing,middle,high,3,no,yes,4,62400
9,937235,sales,junior,medium,1,no,yes,3,26400


,id,quit
0,999029,yes
1,372846,no
2,726767,no
3,490105,no
4,416898,yes
5,223063,no
6,810370,no
7,998900,no
8,578329,no
9,648850,no


,id,job_satisfaction_rate
0,130604,0.74
1,825977,0.75
2,418490,0.60
3,555320,0.72
4,826430,0.08
5,817219,0.76
6,269033,0.64
7,962356,0.38
8,649052,0.14
9,532834,0.59


,id,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary,job_satisfaction_rate
0,155278,sales,junior,medium,2,no,no,1,24000,0.58
1,653870,hr,junior,high,2,no,no,5,38400,0.76
2,184592,sales,junior,low,1,no,no,2,12000,0.11
3,171431,technology,junior,low,4,no,no,2,18000,0.37
4,693419,hr,junior,medium,1,no,no,3,22800,0.20
5,405448,hr,middle,low,7,no,no,4,30000,0.78
6,857135,sales,sinior,medium,9,no,no,3,56400,0.56
7,400657,purchasing,middle,high,9,no,no,3,52800,0.44
8,198846,hr,junior,low,1,no,no,2,13200,0.14
9,149797,technology,middle,high,6,no,no,3,54000,0.47


In [4]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     2000 non-null   int64 
 1   dept                   1998 non-null   object
 2   level                  1999 non-null   object
 3   workload               2000 non-null   object
 4   employment_years       2000 non-null   int64 
 5   last_year_promo        2000 non-null   object
 6   last_year_violations   2000 non-null   object
 7   supervisor_evaluation  2000 non-null   int64 
 8   salary                 2000 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 140.8+ KB


In [5]:
y_test_classif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   int64 
 1   quit    2000 non-null   object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [6]:
y_test_regress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     2000 non-null   int64  
 1   job_satisfaction_rate  2000 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 31.4 KB


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     4000 non-null   int64  
 1   dept                   3994 non-null   object 
 2   level                  3996 non-null   object 
 3   workload               4000 non-null   object 
 4   employment_years       4000 non-null   int64  
 5   last_year_promo        4000 non-null   object 
 6   last_year_violations   4000 non-null   object 
 7   supervisor_evaluation  4000 non-null   int64  
 8   salary                 4000 non-null   int64  
 9   job_satisfaction_rate  4000 non-null   float64
dtypes: float64(1), int64(4), object(5)
memory usage: 312.6+ KB


### Проверим наличие явных дубликатов

In [8]:
def handle_duplicates(dataframe):
    duplicates_count = dataframe.duplicated().sum()

    if duplicates_count > 0:
        dataframe.drop_duplicates(inplace = True)
    
    display(f"Droped {duplicates_count} duplicates.")

In [9]:
handle_duplicates(X_test)
handle_duplicates(y_test_classif)
handle_duplicates(y_test_regress)
handle_duplicates(X_train)

'Droped 0 duplicates.'

'Droped 0 duplicates.'

'Droped 0 duplicates.'

'Droped 0 duplicates.'

### Преобразуем данные к соответствующим типам

In [10]:
X_test = X_test.astype({"dept": "category", "level" : "category", "workload" : "category", "last_year_promo" : "bool", "last_year_violations" : "bool"})
y_test_classif = y_test_classif.astype({"quit": "category"})
X_train = X_train.astype({"dept": "category", "level" : "category", "workload" : "category", "last_year_promo" : "bool", "last_year_violations" : "bool"})

In [11]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     2000 non-null   int64   
 1   dept                   1998 non-null   category
 2   level                  1999 non-null   category
 3   workload               2000 non-null   category
 4   employment_years       2000 non-null   int64   
 5   last_year_promo        2000 non-null   bool    
 6   last_year_violations   2000 non-null   bool    
 7   supervisor_evaluation  2000 non-null   int64   
 8   salary                 2000 non-null   int64   
dtypes: bool(2), category(3), int64(4)
memory usage: 72.9 KB


In [12]:
y_test_classif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      2000 non-null   int64   
 1   quit    2000 non-null   category
dtypes: category(1), int64(1)
memory usage: 17.8 KB


In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     4000 non-null   int64   
 1   dept                   3994 non-null   category
 2   level                  3996 non-null   category
 3   workload               4000 non-null   category
 4   employment_years       4000 non-null   int64   
 5   last_year_promo        4000 non-null   bool    
 6   last_year_violations   4000 non-null   bool    
 7   supervisor_evaluation  4000 non-null   int64   
 8   salary                 4000 non-null   int64   
 9   job_satisfaction_rate  4000 non-null   float64 
dtypes: bool(2), category(3), float64(1), int64(4)
memory usage: 176.4 KB


### Проверим наличие пропусков в данных

In [14]:
X_train.isna().sum()

id                       0
dept                     6
level                    4
workload                 0
employment_years         0
last_year_promo          0
last_year_violations     0
supervisor_evaluation    0
salary                   0
job_satisfaction_rate    0
dtype: int64

Удалим пропуски т.к. их не много.

In [15]:
X_train = X_train[X_train["dept"].notna() & X_train["level"].notna()]
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     3990 non-null   int64   
 1   dept                   3990 non-null   category
 2   level                  3990 non-null   category
 3   workload               3990 non-null   category
 4   employment_years       3990 non-null   int64   
 5   last_year_promo        3990 non-null   bool    
 6   last_year_violations   3990 non-null   bool    
 7   supervisor_evaluation  3990 non-null   int64   
 8   salary                 3990 non-null   int64   
 9   job_satisfaction_rate  3990 non-null   float64 
dtypes: bool(2), category(3), float64(1), int64(4)
memory usage: 207.0 KB


In [16]:
X_test.isna().sum()

id                       0
dept                     2
level                    1
workload                 0
employment_years         0
last_year_promo          0
last_year_violations     0
supervisor_evaluation    0
salary                   0
dtype: int64

Аналогично удалим пропуски в тестовой выборке.

In [17]:
X_test = X_test[X_test["dept"].notna() & X_test["level"].notna()]
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     1997 non-null   int64   
 1   dept                   1997 non-null   category
 2   level                  1997 non-null   category
 3   workload               1997 non-null   category
 4   employment_years       1997 non-null   int64   
 5   last_year_promo        1997 non-null   bool    
 6   last_year_violations   1997 non-null   bool    
 7   supervisor_evaluation  1997 non-null   int64   
 8   salary                 1997 non-null   int64   
dtypes: bool(2), category(3), int64(4)
memory usage: 88.3 KB


In [18]:
y_test_regress.isna().sum()

id                       0
job_satisfaction_rate    0
dtype: int64

In [19]:
y_test_classif.isna().sum()

id      0
quit    0
dtype: int64

### Исправим описки в данных

Проверим данные на наличие описок:

In [20]:
for col in X_train.columns:
    vals = X_train[col].unique()
    display(f"{col}: {vals}")

'id: [155278 653870 184592 ... 533318 706868 338347]'

"dept: ['sales', 'hr', 'technology', 'purchasing', 'marketing']\nCategories (5, object): ['sales', 'hr', 'technology', 'purchasing', 'marketing']"

"level: ['junior', 'middle', 'sinior']\nCategories (3, object): ['junior', 'middle', 'sinior']"

"workload: ['medium', 'high', 'low']\nCategories (3, object): ['medium', 'high', 'low']"

'employment_years: [ 2  1  4  7  9  6  3 10  8  5]'

'last_year_promo: [ True]'

'last_year_violations: [ True]'

'supervisor_evaluation: [1 5 2 3 4]'

'salary: [24000 38400 12000 18000 22800 30000 56400 52800 13200 54000 19200 40800\n 34800 27600 26400 33600 50400 15600 14400 25200 72000 31200 32400 48000\n 43200 46800 58800 84000 44400 39600 37200 21600 28800 62400 60000 42000\n 49200 55200 57600 68400 45600 51600 64800 80400 20400 61200 76800 69600\n 16800 36000 63600 81600 66000 74400 67200 70800 73200 75600 79200 94800\n 78000 88800 92400 85200 91200 98400 96000 97200]'

'job_satisfaction_rate: [0.58 0.76 0.11 0.37 0.2  0.78 0.56 0.44 0.14 0.47 0.74 0.42 0.32 0.57\n 0.16 0.69 0.33 0.64 0.39 0.8  0.79 0.17 0.65 0.18 0.19 0.49 0.63 0.22\n 0.23 0.5  0.35 0.3  0.77 0.88 0.59 0.21 0.36 0.85 0.7  0.48 0.6  0.1\n 0.27 0.71 0.86 0.54 0.73 0.46 0.31 0.72 0.51 0.61 0.81 0.99 0.15 0.91\n 0.68 0.4  0.89 0.67 0.75 0.98 0.26 0.45 0.92 0.82 0.66 0.55 0.38 0.53\n 0.84 0.52 0.24 0.62 0.41 0.28 0.09 0.97 0.83 0.25 0.43 0.04 0.13 0.29\n 0.95 0.93 0.87 0.08 0.94 0.07 0.34 0.9  0.12 0.06 0.96 0.05 1.   0.03]'

В обучающей выборке описок не обранужено, найдены только пропуски, позже обработаем их в пайплайне.

In [21]:
for col in X_test.columns:
    vals = X_test[col].unique()
    display(f"{col}: {vals}")

'id: [485046 686555 467458 ... 305653 160233 771859]'

"dept: ['marketing', 'hr', 'sales', 'purchasing', 'technology', ' ']\nCategories (6, object): ['marketing', 'hr', 'sales', 'purchasing', 'technology', ' ']"

"level: ['junior', 'middle', 'sinior']\nCategories (3, object): ['junior', 'middle', 'sinior']"

"workload: ['medium', 'low', 'high', ' ']\nCategories (4, object): ['medium', 'low', 'high', ' ']"

'employment_years: [ 2  1  5  6  3  9  7  4  8 10]'

'last_year_promo: [ True]'

'last_year_violations: [ True]'

'supervisor_evaluation: [5 4 3 1 2]'

'salary: [28800 30000 19200 40800 42000 26400 49200 62400 14400 36000 33600 44400\n 27600 12000 64800 20400 34800 15600 18000 50400 31200 51600 16800 43200\n 52800 32400 37200 58800 24000 21600 48000 45600 25200 63600 61200 56400\n 38400 46800 55200 13200 39600 54000 57600 68400 22800 72000 81600 60000\n 66000 67200 70800 76800 79200 69600 94800 82800 80400 87600 86400 96000\n 78000 75600 73200 84000 88800 92400]'

В тестовой выборке обнаружены помимо пропусков ещё и пустые строки. Уточним, сколько таких строк.

In [22]:
X_test[(X_test["dept"] == ' ') | (X_test["workload"] == ' ')]["id"].count()

2

Т.к. таких значений всего 2 и важно получить как можно более реалистичные значения оценки качества модели, удалим эти объекты из тестовой выборки вместо попыток замены пропусков.

In [23]:
X_test = X_test[(X_test["dept"] != ' ') & (X_test["workload"] != ' ')]
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1995 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     1995 non-null   int64   
 1   dept                   1995 non-null   category
 2   level                  1995 non-null   category
 3   workload               1995 non-null   category
 4   employment_years       1995 non-null   int64   
 5   last_year_promo        1995 non-null   bool    
 6   last_year_violations   1995 non-null   bool    
 7   supervisor_evaluation  1995 non-null   int64   
 8   salary                 1995 non-null   int64   
dtypes: bool(2), category(3), int64(4)
memory usage: 88.2 KB


### Объединим данные

Объединим все данные в один набор чтобы в последствии избавиться от разницы в распределениях тестовых и обучающих данных и проверить уникальность ID работников в обучающей и тестовой выборках для исключения утечки признаков. 

In [24]:
test = pd.merge(X_test, y_test_regress, on = "id", how = "inner")
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1995 entries, 0 to 1994
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     1995 non-null   int64   
 1   dept                   1995 non-null   category
 2   level                  1995 non-null   category
 3   workload               1995 non-null   category
 4   employment_years       1995 non-null   int64   
 5   last_year_promo        1995 non-null   bool    
 6   last_year_violations   1995 non-null   bool    
 7   supervisor_evaluation  1995 non-null   int64   
 8   salary                 1995 non-null   int64   
 9   job_satisfaction_rate  1995 non-null   float64 
dtypes: bool(2), category(3), float64(1), int64(4)
memory usage: 103.8 KB


Проверим пересечение работников из тестовой и обучающей выборок.

In [25]:
test[test["id"].isin(X_train["id"])]["id"].count()

0

In [26]:
df = pd.concat([test, X_train])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5985 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     5985 non-null   int64   
 1   dept                   5985 non-null   object  
 2   level                  5985 non-null   category
 3   workload               5985 non-null   object  
 4   employment_years       5985 non-null   int64   
 5   last_year_promo        5985 non-null   bool    
 6   last_year_violations   5985 non-null   bool    
 7   supervisor_evaluation  5985 non-null   int64   
 8   salary                 5985 non-null   int64   
 9   job_satisfaction_rate  5985 non-null   float64 
dtypes: bool(2), category(1), float64(1), int64(4), object(2)
memory usage: 391.7+ KB


In [27]:
df = pd.merge(df, y_test_classif, on = "id", how = "left")
df.reset_index(inplace = True, drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5985 entries, 0 to 5984
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     5985 non-null   int64   
 1   dept                   5985 non-null   object  
 2   level                  5985 non-null   category
 3   workload               5985 non-null   object  
 4   employment_years       5985 non-null   int64   
 5   last_year_promo        5985 non-null   bool    
 6   last_year_violations   5985 non-null   bool    
 7   supervisor_evaluation  5985 non-null   int64   
 8   salary                 5985 non-null   int64   
 9   job_satisfaction_rate  5985 non-null   float64 
 10  quit                   1995 non-null   category
dtypes: bool(2), category(2), float64(1), int64(4), object(2)
memory usage: 351.1+ KB


Приведём изменившиеся типы обратно к категориям.

In [28]:
df = df.astype({"dept": "category", "workload" : "category", "quit" : "category"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5985 entries, 0 to 5984
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     5985 non-null   int64   
 1   dept                   5985 non-null   category
 2   level                  5985 non-null   category
 3   workload               5985 non-null   category
 4   employment_years       5985 non-null   int64   
 5   last_year_promo        5985 non-null   bool    
 6   last_year_violations   5985 non-null   bool    
 7   supervisor_evaluation  5985 non-null   int64   
 8   salary                 5985 non-null   int64   
 9   job_satisfaction_rate  5985 non-null   float64 
 10  quit                   1995 non-null   category
dtypes: bool(2), category(4), float64(1), int64(4)
memory usage: 269.6 KB


Был получен и подготовлен файл с чистыми данными.

## Исследовательский анализ данных

### Рассмотрим описательную статистику данных

In [29]:
df.describe()

,id,employment_years,supervisor_evaluation,salary,job_satisfaction_rate
count,5985.000000,5985.000000,5985.000000,5985.000000,5985.000000
mean,547462.127820,3.704929,3.492565,33998.195489,0.538844
std,256606.217838,2.541793,1.005160,15074.023126,0.223658
min,100298.000000,1.000000,1.000000,12000.000000,0.030000
25%,327237.000000,2.000000,3.000000,22800.000000,0.360000
50%,538347.000000,3.000000,4.000000,30000.000000,0.570000
75%,769278.000000,6.000000,4.000000,43200.000000,0.710000
max,999521.000000,10.000000,5.000000,98400.000000,1.000000


- Заметим, что руководители оценивают сотрудников чаще хорошо по пятибальной системе
- Есть сотрудники получающие чуть менее чем в 2 и почти в 3 раза больше средней зарбплаты по компании
- Сотрудники в срекднем работают в компании чуть менее 4 лет
- Распеределение удовлетворённости рабочим местом по описательной статистике походит на нормальное со средним значением чуть выше нейтрального уровня удовлетворённости

Явных выбросов и аномалий при этом не замечено

### Далее построим распределения для количественных признаков.

In [47]:
def plot_numerical_distribution(df: pd.DataFrame, 
                                numerical_feature: str, 
                                target_feature: str = "job_satisfaction_rate",
                                title: str = None):
    quit_vals = df["quit"].dropna().unique()
    val_left = quit_vals[0]
    colors = pd.Series("red", index=df.index)
    colors[df["quit"] == val_left] = "green"
    colors[df["quit"].isnull()] = "grey"
    alphas = pd.Series(0.6, index = df.index)
    alphas[colors == "grey"] = 0.1

    fig, ax = plt.subplots(figsize = (12, 7))
    ax.scatter(df[numerical_feature], df[target_feature], c = colors, alpha = alphas, s = 50, edgecolors = 'w', linewidth = 0.5)
    
    plot_title = title if title else f'Зависимость "{target_feature}" от "{numerical_feature}"'
    ax.set_title(plot_title, fontsize = 16, pad = 20)
    ax.set_xlabel(numerical_feature.replace('_', ' ').capitalize(), fontsize = 12)
    ax.set_ylabel(target_feature.replace('_', ' ').capitalize(), fontsize = 12)
    legend_patches = [
        mpatches.Patch(color = "red", label = "Покинул"),
        mpatches.Patch(color = "green", label = "Остался"),
        mpatches.Patch(color = "grey", label = "Пропуск")
    ]
    ax.legend(handles = legend_patches, title = "Статус сотрудника", loc = "upper right")
    
    plt.show()

In [48]:
plot_numerical_distribution(
    df, 
    "employment_years", 
    title = "Анализ удовлетворенности сотрудников в зависимости от стажа"
)

- Сотрудники, которые работают в компании более 5 лет очень редко катастрофически неудовлетворены своим рабочим местом.
- При этом заметно, что большое количество сотрудников начинают покидать компанию до 3 лет работы в ней.

In [50]:
plot_numerical_distribution(
    df, 
    "supervisor_evaluation", 
    title = "Анализ удовлетворенности сотрудников в зависимости от оценки руководителя"
)

- Сотрудники с низкими оценками руководителей менее всего были довольны своей работой и чаще остальных покидали её.
- Для оценок от 3 и выше наблюдается прямая зависимость между удовлетворённостью сотрудника работой и оценкой его руководителя.
- Заметна логичная особенность "сотрудников-хорошистов" с 3 и 4 баллами по оценкам руководителей: при низкой их удовлетворённостю рабочим местом и относительно высоких оценках сотрудники чаще уходили из компании. При высокой же удовлетворённости и таких же оценках руководства "хорошисты" чащё оставались в компании. Можно предположить, что позитивные оценки руководства мотивируют сотрудников продолжать работать в компании далее, а негативные воспринимаются как токсичность.


In [51]:
plot_numerical_distribution(
    df, 
    "salary", 
    title = "Анализ удовлетворенности сотрудников в зависимости от заработной платы"
)

- Заметно, что сотрудники с высокой удовлетворённостью рабочим местом чаще всего продолжают работу вне зависимости от их оплаты труда
- Низкая и умеренно-высокая удовлетворённость сотрудников работой в компании при условии оплаты труда менее 40_000 у.е. является основанием полагать, что сотрудник прекратит работу в компании
- Заметно, что удовлетворённость сотрудников работой не зависит линейно от оплаты труда

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5985 entries, 0 to 5984
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id                     5985 non-null   int64   
 1   dept                   5985 non-null   category
 2   level                  5985 non-null   category
 3   workload               5985 non-null   category
 4   employment_years       5985 non-null   int64   
 5   last_year_promo        5985 non-null   bool    
 6   last_year_violations   5985 non-null   bool    
 7   supervisor_evaluation  5985 non-null   int64   
 8   salary                 5985 non-null   int64   
 9   job_satisfaction_rate  5985 non-null   float64 
 10  quit                   1995 non-null   category
dtypes: bool(2), category(4), float64(1), int64(4)
memory usage: 269.6 KB


In [ ]:
NUM_BINS = 60  

data = df[["job_satisfaction_rate", "quit"]].dropna(subset = ["job_satisfaction_rate", "quit"])
data["quit_bin"] = data["quit"].map({"no": 0.0, "yes": 1.0})

x_min, x_max = data["job_satisfaction_rate"].min(), data["job_satisfaction_rate"].max()
bins = np.linspace(x_min, x_max, NUM_BINS + 1)
bin_widths = np.diff(bins)
bin_centers = (bins[:-1] + bins[1:]) / 2

bin_idx = np.digitize(data["job_satisfaction_rate"].to_numpy(), bins, right = False) - 1
bin_idx = np.clip(bin_idx, 0, NUM_BINS - 1)

counts = np.zeros(NUM_BINS, dtype = int)
quit_mean = np.full(NUM_BINS, np.nan, dtype = float)

for i in range(NUM_BINS):
    mask = (bin_idx == i)
    if np.any(mask):
        counts[i] = mask.sum()
        quit_mean[i] = data.loc[mask, "quit_bin"].mean()

traffic_cmap = LinearSegmentedColormap.from_list("traffic", ["green", "yellow", "red"], N = 256)
norm = Normalize(vmin = 0.0, vmax = 1.0)

colors = traffic_cmap(norm(np.nan_to_num(quit_mean, nan = 0.5)))  

plt.figure(figsize = (12, 5))
plt.bar(bin_centers, counts, width = bin_widths * 0.98, color = colors, edgecolor = "none")

plt.xlabel("Уровень удовлетворённости (job_satisfaction_rate)")
plt.ylabel("Число сотрудников в корзине")
plt.title("Распределение удовлетворённости с «светофорной» долей увольнений")
plt.grid(axis = "y", linestyle = ":", linewidth = 0.7, alpha = 0.6)

sm = plt.cm.ScalarMappable(norm = norm, cmap = traffic_cmap)
sm.set_array([])
cbar = plt.colorbar(sm, pad = 0.01)
cbar.set_label("Доля уволившихся в корзине (0 → зелёный, 1 → красный)")

plt.tight_layout()
plt.show()


- Удовлетворённость сотрудника раьбочим местом напрямую влияет на прололжение его работы в компании
- Вероятность ухода сотрудника резко падает при удовлетворённости ниже среднего
- График распределения частот удовлетворённости сотрудников мультимодален: неудовлетворённость сотрудников увеливичается до примерно 0.38 и далее падает до нейтрального уровня удовлетворённости. После нейтрального уровня удовлетворённость сотрудников резко повышается до 0.7 и далее так же падает
- Большинство сотрудников умеренно удовлетворены своим рабочим местом

In [ ]:
fig, axs = plt.subplots(ncols = 2, nrows = 1, figsize = (16, 8))

sns.histplot(market, x = "маркет_актив_6_мес", bins = "auto", alpha = 0.5, color = "blue", hue = "покупательская_активность", ax = axs[0])
sns.histplot(market, x = "маркет_актив_тек_мес", bins = "auto", alpha = 0.5, color = "orange", hue = "покупательская_активность", ax = axs[1])
axs[0].set_xlabel("Среднемесячная маректинговая активность за полгода")
axs[0].set_ylabel("Количество пользователей")
axs[1].set_xlabel("Среднемесячная маректинговая активность за последний месяц")
axs[1].set_ylabel("Количество пользователей")
plt.title("Сравнение распределений: 6 последних месяцев vs текущий месяц")
plt.legend()
plt.grid(True)

plt.show()

Частота маркетинговой активности мультимодальна с модой 4 и локальной модой в 5.5 коммуницаций в месяц. Так же заметны значения близкие к выбросам до 2 коммуникаций в месяц. Заметим, что покупательская **активность остаётся на прежднем уровне** во многом у пользователей, которые взаимодействуют **более чем с 5 маркетинговыми активностями в месяц**. При этом так же заметно, что большая часть пользвателей со **сниженным уровнем активности** взаимодействуют с **менее чем 4 маркетинговыми активностями в месяц**.

<div class="alert alert-success">
<h2> Комментарий ревьюера #2 <a class="tocSkip"> </h2>

👍 Теперь все отлично.
</div>


Создадим унифицированную функцию отображения признаков с выделение покупательской активности цветом.

In [ ]:
COLOR_PAIRS = [
    ["#1f77b4", "#ff7f0e"],
    ["#2ca02c", "#d62728"],
    ["#17becf", "#bcbd22"],
    ["#0d6efd", "#fd7e14"],
]

def plot_distribution(df, column, xlabel, title, discrete = False, palette = None):
    fig, ax = plt.subplots(figsize = (10, 6))

    if discrete:
        bins = range(df[column].min(), df[column].max() + 1)
        multiple = "dodge"
        shrink = 0.8
    else:
        bins = 30
        multiple = "layer"
        shrink = 1.0

    if palette is None:
        palette = random.choice(COLOR_PAIRS)

    sns.histplot(df, 
                 x = column, 
                 bins = bins, 
                 hue = "покупательская_активность", 
                 multiple = multiple, 
                 shrink = shrink, 
                 alpha = 0.5, 
                 palette = palette, 
                 ax = ax)

    ax.set_xlabel(xlabel)
    ax.set_ylabel("Количество пользователей")
    ax.set_title(title)
    ax.grid(True)

    plt.show()

In [ ]:
plot_distribution(market, "длительность", "Дней на платформе", "Распределение значений 'длительность'")


Длительность пребывания пользователей на платформе распределена схоже с нормальным распеределением без явных выбросов и аномалий. Заметно, что больше всегшо **вероятность снижения покупательской активности** у пользователей, которые **зарегистрировались на платформе примерно от 7 до 14 месяцев назад**.

<div class="alert alert-warning">
<h2> Комментарий ревьюера  #2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

Хорошо, но тут тоже такая рекомендация - слишком близкие цвета лучше не брать. Хотя и в этом бывают исключения. 

<br/>
<div class="alert alert-info">
Уьрал эти комбинации цветов из случайного выбора. #2
</div>

In [ ]:
plot_distribution(market, "акционные_покупки", "Доля акционных покупок", "Распределение значений 'акционные_покупки'")


Доля акционных покупок распределена мультимодельно. Большинство пользователей покупает лишь треть покупок по акции. Но так же видно меньшее, но значимое, количество пользователей, которые стремятся покупать все товары по акции. Можно сделать однозначный вывод, что пользователи, **покупающие по акции более 35%** покупок с высокой вероятностью **снизят** свою покупательскую **активность** в будущем т.к. таких пользователей меньшенство, но вторая мода в распределении более чем на половину состоит из тких пользователей.

<div class="alert alert-success"  style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера #2 <a class="tocSkip"> </h2>    

👍 Верно.
</div>


<div class="alert alert-success"> 
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

👍 Все верно, у нас есть группа покупателей, которая совершает покупки почти исключительно только по акциям.
    
Здесь можно заметить, что если бы доступ к акциям у всех клиентов был бы одинаковым, то здесь ожидали бы увидеть нормальное или близкое к нему распределение. Если наблюдаем две модельности, можем предположить, что были какие-то специальные предложения для некоторых клиентов.

 

In [ ]:
plot_distribution(market, "средний_просмотр_категорий_за_визит", "Средний просмотр категорий за визит", "Распределение значений 'средний_просмотр_категорий_за_визит'", discrete = True)

Большинство пользователей просматривает от 2 до 5 категорий за визит. При этом явно заметно, что если пользователь **просматривает за раз менее 2 категорий**, у него повышенный риск **снижения активности**.

In [ ]:
plot_distribution(market, "неоплаченные_продукты_штук_квартал", "Количество неоплаченных продуктов за квартал", "Распределение значений 'неоплаченные_продукты_штук_квартал'", discrete = True)

Распределение количества оставленных в корзине без покупки товаров напоминает распределение Пуассона. Чаще всего пользователи оставляют около 2 таваров в корзине в течение 3 месяцев неоплеченными. При том для клиентов со сниженным уровнем активности распределение имеет большую дисперсию и моду. Так, если у пользователя **в корзине более 4 неоплаченных товаров за последние 3 месяца**, то повышается риск **снижения его активности**. 

In [ ]:
plot_distribution(market, "ошибка_сервиса", "Количество ошибок сервиса", "Распределение значений 'ошибка_сервиса'", discrete = True)

Количество ошибок сервиса, с которыми сталкиваются пользователи в течение квартала при посещении сайта распределно нормально со средним значением в 4 ошибки за 3 месяца. При этом у пользователей снижается покупательская активность если они сталкиваются с 2 или 3 ошибками в квартал. Если ошибок 1 или более 4, это мало влияет на покупательскую активность, что странно. В целом количество ошибок вряд ли можно достоверно использовать как предиктор поукпательской активности.

In [ ]:
plot_distribution(market, "страниц_за_визит", "Количество просмотренных страниц за визит", "Распределение значений 'страниц_за_визит'", discrete = True)

Количество просмотренных пользователями с пониженной покупательской активностью страниц за визит в сервис распределно схоже с распределенгием Пуассона. При этом пользователи со стабильной пок3упательской активностью просматривают в мреднем больше страниц за визит и количество страниц распределно ближе к нормальному для стабильных пользователей. Если пользователь **в среднем** просматривает **менее 8 страниз за визит** в течение месяца, у него повышенный риск **снижения активности**.

Т.к. доступны значения выручки для последних 3 месяцев, построим 3 графика и подробно рассмотрим изменения для групп пользователей по целевому признаку.

In [ ]:
market_revenue = pd.pivot_table(market_revenue, values = "выручка", index = "id", columns = "период", aggfunc = "sum")
market_revenue.columns = ["предыдущий_месяц_выручка", "препредыдущий_месяц_выручка", "текущий_месяц_выручка"]
market_revenue.reset_index(inplace = True)
merged_revenue = pd.merge(market, market_revenue, on = "id", how = "inner")

In [ ]:
fig, axs = plt.subplots(ncols = 3, nrows = 1, figsize = (18, 6))

features = [
    ("препредыдущий_месяц_выручка", "Выручка за препредыдущий месяц"),
    ("предыдущий_месяц_выручка", "Выручка за предыдущий месяц"),
    ("текущий_месяц_выручка", "Выручка за текущий месяц"),
]

color_palette = random.choice(COLOR_PAIRS)

for ax, (col, xlabel) in zip(axs, features):
    sns.histplot(merged_revenue, 
                 x = col, 
                 bins = 30, 
                 hue = "покупательская_активность", 
                 alpha = 0.5, 
                 palette = color_palette, 
                 ax = ax)

    ax.set_xlabel(xlabel)
    ax.set_ylabel("Количество пользователей")
    ax.set_title(f"Распределение значений '{col}'")
    ax.grid(True)

plt.tight_layout()
plt.show()


Заметно, что при **выручке от клиента менее 4800 у.е. двумя месяцами ранее** повышается вероятносит **снижения покупательской активности** клиента в будущем. Распределения выручки от пользователей разных целевых категорий за текущий и предыдущий месяца при этом схожи и напоминают нормальное распределение.

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
 
👍 Здесь все верно по фильтрации. Одновременно убрали неактивных и выброс в выручке.
    
Единственное, если покупатель приобрел хотя бы что-то в каждый из месяц, он уже активный. ПОэтому я бы фильтровал прямо по нулю.

In [ ]:
merged_time = pd.merge(market, market_time, on = "id", how = "inner")

plot_distribution(merged_time, "минут", "Минуты сессии пользователя", "Распределение значений 'минут'", discrete = True)

Минуты стабильных клиентов распределены схоже с нормальным распределением со средним в примерно 15 минут проведённых в сервисе за одну сессию. Для клиентов с понижающейся активностью распределение больше напоминает Пуассона и вероятность **понижения покупательской активности** возрастает при значениях **ниже 12 минут за сессию**. 

In [ ]:
merged_profit = pd.merge(market, profit, on = "id", how = "inner")

plot_distribution(merged_profit, "масштабированная_прибыль", "Прибыль от пользователя", "Распределение значений 'прибыль'")

Прибыль распределена схоже с нормальным распределением со средним значением около 4000 у.е. и является плохим предиктором покупательской активности.

### Рассмотрим частоты распределения качестыенных характеристик

In [ ]:
activity_counts = market["покупательская_активность"].value_counts()

plt.figure(figsize = (8, 8))
plt.pie(activity_counts, 
        labels = activity_counts.index, 
        autopct = "%1.1f%%", 
        startangle = 90, 
        colors = plt.cm.Set2.colors)

plt.title("Распределение 'покупательская_активность'")
plt.show()

Примерно у трети пользователей целевой признак покупательской активности снизился в последнее время

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Patch
from matplotlib.colors import to_hex

def plot_categorical_pie(df, category_col, target_col = "покупательская_активность", title = None):
    grouped = df.groupby([category_col, target_col]).size().reset_index(name = "count")
    outer = grouped.groupby(category_col)["count"].sum()
    inner = grouped["count"]
    target_order = ["Снизилась", "Прежний уровень"]
    color_map = {"Снизилась": "#d62728", "Прежний уровень": "#2ca02c"}  # красный / зелёный
    grouped[target_col] = pd.Categorical(grouped[target_col], categories = target_order, ordered = True)
    grouped = grouped.sort_values([category_col, target_col])
    inner_colors = grouped[target_col].map(color_map)
    base_colors = [to_hex(c) for c in plt.cm.Set3.colors]
    forbidden = {"#d62728", "#2ca02c"}
    outer_colors = [c for c in base_colors if c.lower() not in forbidden]
    fig, ax = plt.subplots(figsize = (10, 10))

    ax.pie(outer,
           labels = outer.index,
           radius = 1,
           startangle = 90,
           wedgeprops = dict(width = 0.3, edgecolor = "w"),
           colors = outer_colors[:len(outer)])  # берём столько, сколько категорий

    ax.pie(grouped["count"],
           radius = 0.7,
           startangle = 90,
           labels = None,
           autopct = None,
           wedgeprops = dict(width = 0.3, edgecolor = "w"),
           colors = inner_colors)

    legend_handles = [Patch(facecolor = color_map[val], edgecolor = "w", label = val) for val in target_order]
    ax.legend(handles = legend_handles, title = target_col, loc = "upper right", bbox_to_anchor = (1.2, 1))
    ax.set(aspect = "equal")

    if title is None:
        title = f"Распределение '{category_col}' с долями '{target_col}'"
        
    plt.title(title)
    plt.show()


In [ ]:
plot_categorical_pie(market, "тип_сервиса")

<div class="alert alert-success"  style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера #2 <a class="tocSkip"> </h2>    

👍 Красота. Еще бы знать, а что означает внутренний бублик? нужно как-то это подписать или убрать.
</div>


<br/>
<div class="alert alert-info">
Пояснение в названии графика. Выделил подписи внутренних сегментов в легенду для большей ясности и интуитивности восприятия графика. Использовал интуитивно понятные цвета "хорошо", "плохо". #2
</div>

Чуть менее трети пользователей пользуются подпиской. При этом пользователи, которые **не пользуются подпиской**, имеют более чем **в 2 раза большую вероятность снижения** покупательской **активности**.

In [ ]:
plot_categorical_pie(market, "разрешить_сообщать")

<div class="alert alert-warning">
<h2> Комментарий ревьюера  #2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

Нет, все-таки такая визуализация не будет оптимальной. Для читателя она будет непонятна, подписи у нас не очень хорошо проставлены. Лучше брать столбчатую диаграмму с группировкой.
       
</div>


<br/>
<div class="alert alert-info">
Переделал подписи. Насчёт того, что лучше, вкусовщина. Постарался повысить читаемость легендой и цветом с ассоциацией "хорошо", "плохо". #2
</div>

Около четверти пользователей запретили присылать им маркетинговые сообщения. При этом разрешение на маркетинговые коммуникуации почти ни как не влияет на целевой признак.

In [ ]:
plot_categorical_pie(market, "популярная_категория")

Вероятность **снижения покупательской активности** выше у пользователей, для которых наиболее популярными являются категории:

1) **Косметика и аксесуары**
2) **Кухонная посуда**
3) **Товары для детей**
4) **Домашний текстиль**

Из этого можно сформаировать гиптоезу, что чаще покупательская активность снижается по гендерному признаку у женщин. Требудет дополнительной проверки, можно передать эту информацию в отдел маркетинга.

### Выводы EDA

**Промежуточный вывод**:

Провели ознакомление с данными продаж, очистили данные от выбросов и аномалий, сделали выводы о влиянии признаков на целевой аоказатель активности клиентов.

Клиент, вероятно, снизит свою покупательскую активность если:

1) взаимодествовал менее чем с 4 маркетинговыми активностями в месяц
2) зарегистрирован на платформе примерно от 7 до 14 месяцев назад
3) покупает по акции более 35% товаров
4) просматривает в среднем за раз менее 2 категорий
5) в корзине более 4 неоплаченных товаров за последние 3 месяца
6) просматривает в среднем менее 8 страниз за визит
7) выручка от клиента менее 4800 у.е. двумя месяцами ранее
9) проводит в сервисе менее 12 минут за сессию
10) не пользуется подпиской
11) покупает чаще всего товары из категорий:
- Косметика и аксесуары
- Кухонная посуда
- Товары для детей
- Домашний текстиль

<div class="alert alert-block alert-danger">
<h2>Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
 
Итак, выше не нужно будет удалять неактивных пользователей по длительности. 
  
Тогда у нас будет лишь один выброс и три неактивных пользователя по покупкам. Если развернем таблицу по периодам (Как ты делаешь ниже) получим 1296 записей. В результирующей таблице, соответственно, будем ожидать ровно те же 1296 записей. Так можем себя перепроверить, что все сделали верно. 
    
Пропусков в данных у нас не было, при верном выполнении объединения и предобработки пропуски также не должны будут появляться (за исключением пропусков на месте неактивных пользователей, но это опционально).

<br/>
<div class="alert alert-info">
Всё исправил
</div>


<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
    
Исследовательский анализ мы делаем для того, чтобы понять, какие закономерности заложены в наших данных. Здесь важно посмотреть на распределения признаков в разрезе целевого признака (снижения активности). 
    
Можно построить и наложить друг на друга гистограммы по данным в подвыборках с и без снижения активности. Можно расположить рядом два боксплота, для категориальных данных − две диаграммы.
    
Проще всего наложить гистограммы в контексте таргета, добавив в код гистограмм параметр hue='таргет'.
    
Думаю, так мы увидим гораздо больше интересной информации. Она нам особенно пригодится для части с исследованием сегмента покупателей.

***
    
По итогам исследовательского анализа  у нас должен быть вывод с портретом покупателя, который снижает активность: какие значения каких признаков для него будут характерны. Сколько страниц он в среднем просматривает, больше или меньше это, чем у активных пользователей, также с акционными покупками, временем на сайте и другими признаками. 
    
Фактически основной костяк выводов проекта делаем уже здесь. Модель нам нужна для перевода категориального признака (факт снижения активности) в вероятностный (вероятность снижения), чтобы дальше чуть более пластично исследовать выбранный сегмент.

       
</div> 


<br/>
<div class="alert alert-info">
Всё исправил
</div>

## Объединим таблицы

In [ ]:
merged_profit_revenue = pd.merge(merged_revenue, profit, on = "id", how = "left")

Создадим новые колонки на основании периода для времени сессии.

In [ ]:
market_time = pd.pivot_table(market_time, values = "минут", index = "id", columns = "период", aggfunc = "sum")
market_time.columns = ["минут_предыдущий_месяц", "минут_текущий_месяц"]
market_time.reset_index(inplace = True)

In [ ]:
merged = pd.merge(merged_profit_revenue, market_time, on = "id", how = "inner")

In [ ]:
merged.info()

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 
 

Здесь неоптимально объединяем таблицы. Есть способ, как в 4-5 строк и отфильтровать активных пользователей, и все объединить. 
    
Используем сводные таблицы, развернув наши таблицы с периодами так, чтобы в строках остались уникальные id, а в столбцах были периоды. Дальше просто отбираем строки, где ни в одном из столбцов нет нулей, после чего просто парой строк объединяем наши три таблицы - они уже все в готовом для этого формате.

</div>


<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
    
Важно корректно обработать данные и корректно «развернуть» таблицы. При объединении стоит использовать inner, ведь мы не хотим, чтобы в результирующую таблицу попали строки, которые ранее мы удаляли. 
    
Еще раз: в итоге будет 1296 строк. Сейчас некоторые строки у тебя задублированы. Соответственно, последующий анализ и моделирование будут показывать некорректные результаты.

<br/>
<div class="alert alert-info">
Всё исправил
</div>

### Заполним пропуски в новых признаках

Чтобы не искажать статистику заполним пропуски средними.

In [ ]:
for col in merged.select_dtypes(include = ["number"]).columns:
    mean_value = merged[col].mean()
    merged[col].fillna(mean_value, inplace = True)

merged.info()

## Корреляционный анализ

### Корреляционная матрица

Для начала построим кореляционную матрицу в виде тепловой карты.

In [ ]:
numeric_columns = merged.select_dtypes(include = ["number"]).columns.tolist()
numeric_columns.remove("id")
fig, ax = plt.subplots(figsize=(10,10))  
display(numeric_columns)
phi_k = merged[numeric_columns].phik_matrix()
sns.heatmap(phi_k, cmap = "YlGnBu", annot = True)
ax.set_title("Зависимость всех признаков между собой")
plt.show()

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
    
Давай поправим график так, чтобы числа не налезали друг на друга. Пока график нечитаем.

<br/>
<div class="alert alert-info">
Увеличил размер графика
</div>

### VIF (коэффициент инфляции дисперсии)

Оценим мультиколлинеарность каждого не содержащего пропусков численного признака в данных при помощи рассчёта коэффициента инфляции дисперсии

In [ ]:
X = add_constant(merged[numeric_columns])
VIFs = pd.DataFrame()
VIFs["Variable"] = X.columns
VIFs["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

display(VIFs.sort_values(by = "VIF", ascending = False))

Исходя из корелляционной матрицы и анализа VIF делаем вывод, что мультиколлинеарность в данных почти отсутствует. Наиболее скоррелированным с другими признаками является выручка за предыдущий и текущий месяц. Оставим все признаки в силу их важности.

In [ ]:
preproc_merged = merged.drop("предыдущий_месяц_выручка", axis = 1)

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
    
Если мультиколлинеарности нет, а признак хорошо коррелирует с таргетом, удалять такой признак − очень плохая идея. Этот признак для нас может быть самым важным, без него качество будет хуже.

<br/>
<div class="alert alert-info">
Оставил признаки
</div>

## Построение пайплайнов

### Подготовка данных

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    preproc_merged.drop(["id","покупательская_активность"], axis=1),
    preproc_merged["покупательская_активность"],
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = preproc_merged["покупательская_активность"])

ohe_columns = ["тип_сервиса", "популярная_категория"]
numeric_columns = preproc_merged.select_dtypes(include = ["number"]).columns.tolist()
numeric_columns.remove("id")
numeric_columns.remove("прибыль")

ohe_pipe = Pipeline(
        [("ohe", OneHotEncoder(drop = "first", handle_unknown = "error", sparse = False))]
    )

numeric_pipe = Pipeline(
    [("num", MinMaxScaler())]
    )

data_preprocessor = ColumnTransformer(
    [("ohe", ohe_pipe, ohe_columns),
     ("num", MinMaxScaler(), numeric_columns)
    ], 
    remainder = "passthrough"
)

### Cоздаём итоговый пайплайн: перебор методов подготовки данных и моделей сразными гиперпараметрами

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 11),
        'models__max_features': range(2, 11),
        'models__min_samples_split': range(2, 11, 2),
        'models__min_samples_leaf': range(1, 6),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), MaxAbsScaler(), 'passthrough']  
    },
    {
        "models": [KNeighborsClassifier(n_jobs = -1)],
        "models__n_neighbors": range(2, 7),
        "models__weights": ["uniform", "distance"],
        "models__algorithm": ["ball_tree", "kd_tree"],
        "models__leaf_size": range(2, 51, 3),
        "models__p": [1, 2],
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), RobustScaler(), MaxAbsScaler(), "passthrough"]   
    },
    {
        "models": [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver="liblinear",
            n_jobs = -1
        )],
        "models__solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
        "models__penalty": ["l1", "l2", "elasticnet"],
        "models__penalty": ["l1", "l2", "elasticnet"],
        "models__C": [1, 2, 4, 8, 16, 32],
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), RobustScaler(), MaxAbsScaler(), "passthrough"]  
    },
    {
        "models": [SVC(
            random_state=RANDOM_STATE
        )],
        "models__C": [1, 2, 4, 8, 16, 32],
        "models__kernel": ["linear", "poly", "rbf", "sigmoid"],
        "models__degree": [2, 3, 4, 5],
        "models__gamma": ["scale", "auto"],
        "preprocessor__num": [StandardScaler(), MinMaxScaler(), RobustScaler(), MaxAbsScaler(), "passthrough"]  
    }
]

randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_grid,
    n_iter = 30,
    cv = 5,
    scoring = "roc_auc",
    random_state = RANDOM_STATE,
    n_jobs = -1
)

randomized_search.fit(X_train, y_train)

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

👍 Отлично, что используешь `remainder='passthrough'` . Хороший выбор гиперпараметров и значений для подбора. Отлично, что пробуем два типа масштабирования.
</div> 



### Выбор метрики

Для задачи выбрана метрика ROC AUC т.к. она более робастная, подходит для задачи бинарной классификации и позволяет объективнее оценить обобщающую способность разных моделей вне зависимости от их гиперпараметров.

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

👍 Все так. Конркетно в нашей задаче нам важно лишь то, что эта метрика учитывает и полноту, и точность комплексно, а не по отдельности.
    
Дисбаланс классов у нас небольшой, порог нам подбирать не нужно, поэтому в целом подошли бы все комплексные метрики : f1, accuracy, ROC_AUC
</div> 



In [ ]:
display("Лучшая модель и её параметры:\n\n", randomized_search.best_estimator_)
display ("Метрика лучшей модели на тренировочной выборке:", randomized_search.best_score_)

y_test_pred = randomized_search.predict_proba(X_test)[:,1]

display(f"Метрика ROC-AUC на тестовой выборке: {roc_auc_score(y_test, y_test_pred)}")

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

По вычислениям верно, по результату неадекватно, так как у нас задублировались строки и соответственно произошла утечка таргета.

<br/>
<div class="alert alert-info">
Получил валидную модель с адекватной точностью и орбновил вывод
<div>

### Вывод подбора модели

**Наилучшей** моделью оказалась **модель ближайших соседей** с количеством рассматритваемых соседей 4 и расстоянием считаемым по манхеттенской метрике со значением метрики **ROC AUC близким к 0.88** на тестовой выборке. Такое значение точности выглядит правдоподобным и удовлетворительным

## Анализ важности признаков

Для анализа значимости признаков рассчитаем значения Шепли. Семплируем тестовую выборку для адекватного соотношения времени ожидания перебора параметров, репрезентативности выборки и стабильности резултатов.

<br/>
<div class="alert alert-info">
Из-за нестабильности результатов вынужден был увеличить размер выборки. Пробовал 20, 30, 100, 200, 300 #3
<div>

!pip install matplotlib==3.5.2 -q

!pip install scikit-learn==1.1.3 -q

In [ ]:
def model_predict(x):
    return randomized_search.best_estimator_.predict_proba(x)[:, 1]

sampled_train = shap.sample(X_train, 500, random_state = RANDOM_STATE)
sampled_test = shap.sample(X_test, 500, random_state = RANDOM_STATE)
explainer = shap.KernelExplainer(model_predict, sampled_train, keep_index = True, keep_index_ordered = True)
shap_values = shap.Explanation(explainer.shap_values(sampled_test), feature_names = sampled_test.columns)

<br/>
<div class="alert alert-info">
Получил валидную модель с адекватной точностью и обновил вывод
<div>

### График влияния признаков на предсказания классов

<div class="alert alert-warning">
<h2> Комментарий ревьюера  #2 <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

ВЫше почему-то график не выглядит информативным. Я бы предположил, что проблема в алгоритме knn, у которого своеобразные отношения с «вероятностями классов». 
    
***
    
Давай проговорим по shap 
    
Сам Shap строится без проблем в пару строк, если в него подать обычный датафрейм с подписанными колонками. Однако у нас здесь пайплайн. Поэтому нужно приложить некоторые усилия для того, чтобы вынуть из пайплайна названия фичей и подать в функцию графика в нужном нам виде датафрейма.
    
    
По Shap давай попробуем такой код:
    
```
    

X_train_2 = pipe_final.named_steps['preprocessor'].fit_transform(X_train)
 
explainer = shap.Explainer(randomized_search.best_estimator_.named_steps['models'], X_train_2)
 
X_test_2 = pipe_final.named_steps['preprocessor'].transform(X_test)
 
feature_names = pipe_final.named_steps['preprocessor'].get_feature_names_out()

X_test_2 = pd.DataFrame(X_test_2, columns=feature_names)
 
shap_values = explainer(X_test_2)
 
display(shap.plots.bar(shap_values, max_display=22))
shap.plots.beeswarm(shap_values, max_display=22) 


```
 
По идее, нам просто нужно подать в функцию shap.Explainer() модель и тренировочную выборку, а затем в функцию отрисовки графика shap.plots.bar() эксплейнер с тестовой выборкой explainer(X_test_2). 
    
Однако так как мы используем пайпланый, важно подавать в эксклейнер тренировочную выборку после масштабирования и кодирования Поэтому мы берем из пайплайна только соответствующие шаги и применяем к тренировочной выборке.
    
`X_train_2 = pipe_final.named_steps['preprocessor'].fit_transform(X_train)`
    
То же самое делаем с тестовой выборкой. 
    
`X_test_2 = pipe_final.named_steps['preprocessor'].transform(X_test)`
    
И отдельным шагом сохраняем список с названиями столбцов:
    
`feature_names = pipe_final.named_steps['preprocessor'].get_feature_names_out()`
    
После чего получаем датасет с колонками после масштабирования и экнодинга и их названиями.
    
`X_test_2 = pd.DataFrame(X_test_2, columns=feature_names)`
    
Дальше оставется все это использовать:
    
```
shap_values = explainer(X_test_2)
 
display(shap.plots.bar(shap_values, max_display=22))    
    
```

Код выше предполагает, что мы подаем в эксплейнер «знакомую» для shap модель. Согласно документации, модели из пакета sklearn должны быть знакомы, но на практике иногда shap не хочет работать с частью моделей, например, как раз с SVC. Тогда мы используем классическое решение, где в эксплейнер подается метод предикт (в зависимости от задачи). Для задачи классификации мы должны предсказать вероятности, а дальше взять по индексу веротность класса «1».



```
# преобразуем данные обучающей выборки
X_train =  pipe_final.named_steps['preprocessor'].fit_transform(X_train)
X_summary = shap.sample(X_train, nsamples=50)

# создадим объект explainer
explainer = shap.Explainer(grid_1.best_estimator_.named_steps['models'].predict_proba, X_summary)

# преобразуем данные тестовой выборки
X_test = pipe_final.named_steps['preprocessor'].transform(X_test)

# получим имена признаков
feature_names = pipe_final.named_steps['preprocessor'].get_feature_names_out()

# создадим датафрейм для тестовых данных с именами признаков
X_test = pd.DataFrame(X_test, columns=feature_names)

# посчитаем значения SHAP для тестовых данных
shap_values = explainer(X_test)

# визуализируем значения SHAP в виде столбчатой диаграммы
shap.plots.bar(shap_values[..., 1])
shap.plots.beeswarm(shap_values[..., 1], max_display=16)
```


<br/>
<div class="alert alert-info">
Ваш код не работает. Нужно передавать в fit_transform пайплайна df, но у нас массив векторов numpy. Я долго с этим возился и пришёл к такому результату. Нет времени возиться с этим дальше т.к. есть реальные проекты с реальными заказчиками, которые приоритетнее.

Я не первый, кто с этим столкнулся. Библиотека не работает корректно с пайплайнами и чтобы это починить нужно залазить в ядро их кода, метод shap.Explainer. Я в рамках обучающего проекта этим заниматься не планирую.
https://github.com/shap/shap/issues/1058

#2
</div>

<div class="alert alert-block alert-danger" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера #3 <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>

Да, в некоторых случаях возникает эта проблема. Это действительно известный баг. 

Проблема с fontsize решается так:
    
```
    
!pip install scikit-learn==1.1.3 -q
!pip install matplotlib==3.5.2 -q
    
    
```
    
Хорошо, ты можешь использовать абсолютно любой из вариантов, предложенный в тренажере, в официальной документации или в моих примерах. Однако нам важно построить график, который будет корректно отражать важность признаков и корректно отрабатывать в нашей среде. 
    
Моя задача как ревьюера на этом моменте проверить, что ты выполнил задание по построению графика важности признаков shap, график корректен и отражает действительную важность признаков, а дальше ты проанализировал этот график и пришел к верным выводам. Пока у нас несостыковка с тем, что твой график beeswarm показывает не то, что должен показывать.
    
Если у тебя возникают технические сложности с графиком, всегда можешь обратиться к наставнику в Пачке или ко мне − обязательно подскажем. 
       
</div>


<br/>
<div class="alert alert-info">
Beeswarm построить так и не получилось. Доходил до шага, в котором в класс Explainer получалось передать df с подготовленными данными и преобразованными названиями фич, но всё равно конструктор выдавал ошибку с непонятной сутью, о том, что значения должны быть числовыми и меньше беспонечности. Я скаслил все типы в подготовленном после препроцессинга df ко float, но не помогло. Дальше перестал пробовать. Вероятно, нужно распотрошить pipeline и попробовать скастить вручную все поля к таким же типам как и после preprocessing шага, но у меня нет на это ни времени ни желания. 

Наставник после вопроса, что с этим можно сделать, дал общие рекомендации и предположил, что если получится построить корректный график важности фич любым способом с использованием библиотеки SHAP и его верно проинтерпретировать, то этот шаг проекта будет принят.

Я заметил, что при семплировании того количества объектов, которое я использовал для вменяемого времени выполнения алгоритма, не получается для стабильной статистики. Пришлось увеличивать значение обхектов в выборке. K-means оптимизация не работает т.к. у меня не все фичи числовые изначално. Для pipeline'а это не проблема, а вот классы KernelExplainer как и обычный, более удобный, Explainer, не могут рассчитать значения Шепли с исходными обучающей и тестовой выборками даже если в качестве модели передать весь pipeline с preprocessing'ом.

В итоге остановился на выборке из 500 значений, которая у меня локально считается за 2 часа и базовом графике со столбчатой диаграмой важности фич. Надеюсь, этого объективно достаточно чтобы оценить важность каждого из исходных признаков. В загруженном ноутбуке должны быть закешиованы результаты выполнения. На всякий случай, если их нет, т.к. я понимаю, что 2 часа ждать никто не будет, прикладываю ссылку к моему публичному GitHub репозиторию, где точно можно посмотреть закешированные результаты рассчёта значений Шепли т выводы:
https://nbviewer.org/github/EvanderLatine/DS_Practicum/blob/252b90b66d4c99454622ce2ff747604254777f4d/Customers_Retention/research.ipynb

#3
</div>

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

Здесь также пока график ничего не показывает, но по коду ок.

- В силу масштабирования данных в ходе предобработки все признаки имеют примерно одинаковые диапазоны значений, что отражено монохромностью точек на графике. 

- Часто конкретные признаки никак не влияют на предсказание модели.

- В более редких случаях, модель чаще склоняется к тому, что пользователь сохранит свою покупательскую активность.

- Заметно, что при снижении покупательской активности пользователей, модель "увереннее" присваивает класс на основании любого из признаков.

- При снижении покупательской активности модель незначительно больше учитывает более значимые признаки, чем менее значимые.

<br/>
<div class="alert alert-info">
Пересмотрел заметки
</div>

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

Важно все будет перепроверить и переписать выводы.

### График значимости признаков

In [ ]:
shap.plots.bar(shap_values, max_display = 18)

- Наиболее важными для предсказания покупательской активности являются прибыль от него, количество акционных покупок и выручка от него в этом месяце.

- Значимость остальных признаков равномерно убывает, но при этом остаётся достаточно существенной, чтобы их оставить.

- Наименее значимыми оказались средняя продолжительность сессии клиента и количество просматреваемых за визит категорий.

### Выводы анализа влияния признаков

Стоит сосредоточиться на:

- Финансовых показателях пользователей

- Количестве невыкупленных товаров для повышения удержания клиентов

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 
    
Важно все уточнить.

<br/>
<div class="alert alert-info">
Обновил выводы. Для стабилизации значений Шепли увеличил размер выборки примерно до половины всего доступного объёма данных. #3
</div>

## Сегментация покупателей

Применяем модель ко всему набору данных для получения оценок пользователйской активности модели

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

В самом начале было бы логично прописать, како сегмент и почему мы выбрали для анализа.
    
Логика может быть следующей: выше мы исследовали, чем активные клиенты различаются с клиентами, чья активность снижается. Проанализировали все закономерности итд, а здесь мы на основе анализа выбираем группу, активность которой нам выгоднее удерживать в первую очередь. Или группу, исследование которой может уточнить наши выводы о данных.



<br/>
<div class="alert alert-info">
Это уже было сделано, соответствующий код и график ниже.
</div>

In [ ]:
X_full = preproc_merged.drop(["id", "покупательская_активность"], axis = 1)

predictions = randomized_search.best_estimator_.predict_proba(X_full)[:, 1]

segmented = merged.copy()
segmented["нужно_удержание"] = predictions

Рассмотрим распределение вероятностей необходимости удержания клиентов.

In [ ]:
plt.hist(predictions)

Видно, что модель присваивает абсолютные значения вероятности для большинства клиентов.

Сегментируем покупателей по прибыли от них и необходимости действой по удержанию.

In [ ]:
clustering_features = ["масштабированная_прибыль", "нужно_удержание"]
X_cluster = segmented[clustering_features].fillna(segmented[clustering_features].mean())
scaler = StandardScaler()
X_cluster_scaled = scaler.fit_transform(X_cluster)

kmeans = KMeans(n_clusters=4, random_state=RANDOM_STATE)
segmented['segment'] = kmeans.fit_predict(X_cluster_scaled)


In [ ]:
segmented.describe()

Описательная статистика выглядит адекватно. Явных аномалий не замечено.

### Квадранты сегментации клиентов

Разделение по вероятности снижения активности клиента будем делать по 0.5 т.к. логично рассматривать этот признак как бинарный. 

Для значений прибыльности клиентов сделаем раздиление по средней прибыльности т.к. ранее мы наблюдали нормальное распределение для этого признака.

In [ ]:
retention_threshold = 0.5
profit_threshold = segmented["масштабированная_прибыль"].mean()

display(f"Порог для удержания: {retention_threshold}")
display(f"Порог для прибыли: {profit_threshold:.2f}")
display(f"Количество пользователей в каждом сегменте:")
display(segmented["segment"].value_counts())

plt.figure(figsize = (12, 10))

colors = ["red", "blue", "green", "orange"]
segment_names = ["Сегмент 1", "Сегмент 2", "Сегмент 3", "Сегмент 4"]

for segment_id in range(4):
    segment_data = segmented[segmented["segment"] == segment_id]
    plt.scatter(segment_data["нужно_удержание"],
                segment_data["масштабированная_прибыль"],
                c = colors[segment_id],
                alpha = 0.6,
                s = 50,
                label = f"{segment_names[segment_id]} (n = {len(segment_data)})")

plt.axhline(y = profit_threshold, color = "black", linestyle = "--", linewidth = 2, alpha = 0.7)
plt.axvline(x = retention_threshold, color = "black", linestyle = "--", linewidth = 2, alpha = 0.7)

plt.xlabel("Вероятность необходимости удержания", fontsize = 12, fontweight = "bold")
plt.ylabel("Прибыль (у.е.)", fontsize = 12, fontweight = "bold")
plt.title("Сегментация пользователей: Удержание vs Прибыль", fontsize = 14, fontweight = "bold")



plt.legend(bbox_to_anchor = (1.05, 1), loc = "upper left")
plt.grid(True, alpha = 0.3)


plt.tight_layout()
plt.show()

Для бизнеса в первую очередь стоит сфокусировать внимание на клиентах с высокой прибыльностью, которые теряют активность. На графике это 3 сегмент, обозначенный зелёным цветом.

### Анализ распределения по квадрантам

In [ ]:
def get_quadrant(row):
    retention = row["нужно_удержание"]
    profit = row["масштабированная_прибыль"]

    if retention >= retention_threshold and profit >= profit_threshold:
        return "Квадрант 1: Нужно удержание + Высокая прибыль"
    elif retention < retention_threshold and profit >= profit_threshold:
        return "Квадрант 2: Не нужно удержание + Высокая прибыль"
    elif retention < retention_threshold and profit < profit_threshold:
        return "Квадрант 3: Не нужно удержание + Низкая прибыль"
    else:
        return "Квадрант 4: Нужно удержание + Низкая прибыль"

segmented["quadrant"] = segmented.apply(get_quadrant, axis=1)

quadrant_counts = segmented["quadrant"].value_counts()
display("Распределение пользователей по квадрантам:")
display(quadrant_counts)
display("\nПроцентное распределение:")
display(f"{(quadrant_counts / len(segmented) * 100).round(1)} %")

Пользователей, на которых в первую очередь стоит обратить внимание в текущем наборе данных примерно пятая часть.

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
 
`8.1 Выполните сегментацию покупателей. Используйте результаты моделирования и данные о прибыльности покупателей.`

Попробую описать, что мы должны сделать:
    
Самое важное − мы должны по заданию выбрать некоторый сегмент пользователей, обосновать выбор сегмента, обосновать то, как мы этот сегмент определяем (почему выбираем такие значения признаков для отбора пользователей в сегмент), а дальше исследовать только этот сегмент.
    
1) Под результатами моделирования здесь мы понимаем предсказания нашей модели, то есть, вероятности классов. Мы можем ранжировать клиентов по вероятности снижения активности и таким образом использовать эту информацию как одну из осей скаттерплота (для примера). Второй осью тогда будет какая-то категория, которая будет логичной после выбора сегмента. Например, если выбираем сегмент с высокой вероятностью снижения активности и высокой выручкой, то второй шкалой в скаттерплоте будет выручка.

    
2) Важно аргументировать выбор границ и для вероятности снижения активности, и для прибыльности. Как раз это будет удобно сделать, солавшись на график (думаю, скаттерплот тут в качестве типа визуализации будет выигрывать).
    
    
Дальше следует провести исследование для выбранного сегмента: посмотреть на данные в разрезе периодов, в разрезе категорий товаров, акций итд. Нужно выявить факторы, которые сильнее всего влияют на снижение активности и предложить решения для минимизации негативного воздействия этих факторов.

</div>





In [ ]:
premium = segmented[segmented["quadrant"] == "Квадрант 1: Нужно удержание + Высокая прибыль"]

<br/>
<div class="alert alert-info">
Сегмент для исследования выбран в соответствии с заданием данного раздела проекта
</div>

### Идеи удержания пользователей

Проверим возможности предоставить пользователям скидки.

In [ ]:
promo_active_threshold = 0.2

promo_users = premium[premium["акционные_покупки"] > promo_active_threshold]
non_promo_users = premium[premium["акционные_покупки"] <= promo_active_threshold]

promo_counts = [len(promo_users), len(non_promo_users)]
promo_labels = [
    f"Активно пользуются акциями\n({len(promo_users)} чел., {len(promo_users)/len(premium)*100:.1f}%)",
    f"Редко/не пользуются акциями\n({len(non_promo_users)} чел., {len(non_promo_users)/len(premium)*100:.1f}%)"
]

plt.figure(figsize = (10, 8))
colors = ["#FF6B6B", "#4ECDC4"]

plt.pie(promo_counts,
        labels = promo_labels,
        colors = colors,
        autopct = "%1.1f%%",
        startangle = 90,
        explode = (0.1, 0),
        shadow = True,
        textprops = {"fontsize": 12, "fontweight": "bold"})

plt.title("Использование акционных покупок в приоритетном сегменте\n(Высокая прибыль + Высокая необходимость удержания)",
          fontsize = 14, fontweight = "bold", pad = 20)

plt.axis("equal")
plt.tight_layout()
plt.show()

Пользователям, которые совершают менее 20% покупок по акции можно чаще предоставлять скидки. Но таких пользователей меньшенство, около 10%, что не приведёт к значительному повышени. удержания клиентов.

Проверим количество ошибок сервиса, с которыми сталкиваются премиальные пользователи.

In [ ]:
def categorize_service_errors(errors_count):
    if errors_count == 0:
        return "Без ошибок"
    elif errors_count <= 2:
        return "1-2 ошибки"
    elif errors_count <= 5:
        return "3-5 ошибок"
    else:
        return "6+ ошибок"

premium["service_error_category"] = premium["ошибка_сервиса"].apply(categorize_service_errors)
error_categories = premium["service_error_category"].value_counts().sort_index()

plt.figure(figsize = (12, 9))

colors = ["#2E8B57", "#FFD700", "#FF6347", "#DC143C"]

plt.pie(error_categories.values,
        labels = [f"{cat}\\n({count} чел., {count/len(premium)*100:.1f}%)"
                for cat, count in error_categories.items()],
        colors = colors,
        autopct = "%1.1f%%",
        startangle = 90,
        shadow = True,
        textprops = {"fontsize": 11, "fontweight": "bold"})

plt.title("Распределение ошибок сервиса в приоритетном сегменте\n(Высокая прибыль + Высокая необходимость удержания)",
          fontsize = 14, fontweight = "bold", pad = 20)

stats_text = f"Среднее количество ошибок: {premium['ошибка_сервиса'].mean():.2f}\n" \
             f"Медиана: {premium['ошибка_сервиса'].median():.0f}\n" \
             f"Максимум: {premium['ошибка_сервиса'].max()}\n" \
             f"Всего пользователей: {len(premium)}"

plt.text(-1.8, -1.3, stats_text,
         fontsize = 10, bbox = dict(boxstyle = "round,pad=0.5", facecolor = "lightblue", alpha = 0.8))

plt.axis("equal")
plt.tight_layout()
plt.show()

Для премиальных пользователей количество ошибок не сильно отличается от общего распределения для всех клиентов. Вряд ли можно как-то повлиять на их удержание исходя из этого показателя.

Рассмотрим распределение премиальных пользователей по типам сервиса.

In [ ]:
service_type_counts = premium["тип_сервиса"].value_counts()

plt.figure(figsize = (10, 8))

colors = ["#FFD700", "#C0C0C0"]

plt.pie(service_type_counts.values,
        labels = [f"{service_type}\n({count} чел., {count/len(premium)*100:.1f}%)"
                for service_type, count in service_type_counts.items()],
        colors = colors,
        autopct = "%1.1f%%",
        startangle = 90,
        explode = (0.1, 0),
        shadow = True,
        textprops = {"fontsize": 12, "fontweight": "bold"})

plt.title("Распределение типов сервиса в приоритетном сегменте\n(Высокая прибыль + Высокая необходимость удержания)",
          fontsize = 14, fontweight = "bold", pad = 20)

plt.axis("equal")
plt.tight_layout()
plt.show()

Среди премиальных клиентов схожее с общим распределение покупки подписок. Только треть пользуется подпиской на сервис. **Можно продумать специальное предложение подписки на сервис премиальным клиентам для повышения их удержания**, т.к. такой шаг будет актуален для большинства премиальных клиентов в силу отсутствия у них подписки.

Проанализируем популярные среди премиальных пользователей категории и среднюю прибыльность для бизнеса каждой из них.

In [ ]:


def plot_category_pie_with_profit_gradient(
    df: pd.DataFrame,
    category_col: str = "популярная_категория",
    profit_col: str = "масштабированная_прибыль",
    top_n: int = 8,
    title: str = "Популярные категории товаров в приоритетном сегменте\n(Высокая прибыль + Высокая необходимость удержания)",
    yellow_hex: str = "#F7DC6F",
    green_hex: str = "#2ECC71",
    others_label: str = "Другие",
    others_color: str = "#B0BEC5",
    explode_value: float = 0.05
):
    category_counts = df[category_col].value_counts()
    top_categories = category_counts.head(top_n)
    other_count = int(category_counts.iloc[top_n:].sum())
    has_others = other_count > 0

    if has_others:
        top_categories[others_label] = other_count

    profit_by_cat = (
        df[[category_col, profit_col]]
        .groupby(category_col)[profit_col]
        .mean()
    )
    real_top_cats = [c for c in top_categories.index if c != others_label]
    real_means = profit_by_cat.reindex(real_top_cats)
    vmin = float(np.nanmin(real_means.values)) if len(real_means) else 0.0
    vmax = float(np.nanmax(real_means.values)) if len(real_means) else 1.0

    if np.isclose(vmin, vmax):
        vmin -= 0.5
        vmax += 0.5

    norm = Normalize(vmin = vmin, vmax = vmax)
    cmap = LinearSegmentedColormap.from_list("profit_cmap", [yellow_hex, green_hex])

    colors = []
    for cat in top_categories.index:
        if cat == others_label:
            colors.append(others_color)
        else:
            mean_profit = profit_by_cat.get(cat, np.nan)
            colors.append(cmap(norm(mean_profit)))

    total = len(df)
    labels = []
    
    for cat, count in top_categories.items():
        pct = 100 * count / total if total > 0 else 0
        if cat == others_label:
            labels.append(f"{cat}\n({count} чел., {pct:.1f}%)")
        else:
            mean_profit = profit_by_cat.get(cat, np.nan)
            profit_txt = "н/д" if pd.isna(mean_profit) else f"{mean_profit:,.2f}".replace(",", " ")
            labels.append(f"{cat}\n({count} чел, {pct:.1f}% | ср. прибыль: {profit_txt})")

    explode = [explode_value] * len(top_categories)

    fig, ax = plt.subplots(figsize=(18, 10))
    wedges, texts, autotexts = ax.pie(
        top_categories.values,
        labels = labels,
        colors = colors,
        autopct = "%1.1f%%",
        startangle = 90,
        explode = explode,
        shadow = True,
        textprops = {"fontsize": 10, "fontweight": "bold"},
        wedgeprops = dict(linewidth = 1, edgecolor = "white")
    )
    ax.set_title(title, fontsize = 14, fontweight = "bold", pad = 20)
    ax.axis("equal")

    sm = ScalarMappable(norm = norm, cmap = cmap)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax = ax, fraction = 0.046, pad = 0.04)
    cbar.set_label("Среднее значение на пользователя (по категории) в у.е.", fontsize = 11)

    plt.show()

In [ ]:
plot_category_pie_with_profit_gradient(
    df = premium,
    category_col = "популярная_категория",
    profit_col = "масштабированная_прибыль",
    top_n = 8
)

Схожие со всеми остальными пользователями значения популярности категорий товаров. Заметно, какие категории товаров в премиальном сегменте пользователей приносят наибольшую прибыльность компании:
1) Косметика и аксессуары
2) Товары для детей и дома
3) Техника для красоты и здоровья

Далее рассмотрим на динамику выручки в разных категориях товаров от премиальных пользователей по последним месяцам.

In [ ]:
plot_category_pie_with_profit_gradient(
    df = premium,
    category_col = "популярная_категория",
    profit_col = "препредыдущий_месяц_выручка",
    top_n = 8,
    title = "Популярные категории товаров в приоритетном сегменте с выручкой за препредыдущий месяц\n(Высокая прибыль + Высокая необходимость удержания)"
)

In [ ]:
premium.info()

In [ ]:
plot_category_pie_with_profit_gradient(
    df = premium,
    category_col = "популярная_категория",
    profit_col = "предыдущий_месяц_выручка",
    top_n = 8,
    title = "Популярные категории товаров в приоритетном сегменте с выручкой за предыдущий месяц\n(Высокая прибыль + Высокая необходимость удержания)"
)

In [ ]:
plot_category_pie_with_profit_gradient(
    df = premium,
    category_col = "популярная_категория",
    profit_col = "текущий_месяц_выручка",
    top_n = 8,
    title = "Популярные категории товаров в приоритетном сегменте с выручкой за текущий месяц\n(Высокая прибыль + Высокая необходимость удержания)"
)

- Стабильно высокая выручка от премиальных пользователей в сегментах кухонной посуды и косметики и аксессуаров
- За последние два месяца упала выручка в сегментах техники для красоты и здоровья, товаров для детей
- За последние 2 месяца повысилась выручка в сегментах мелкой бытовой техники и электроники, домашнего текстиля


<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

Сейчас смотришь категории по всем периодам. Хорошо бы посмотреть разницу в выручке между периодами для разных категорий. Так увидим, в какой категории динамика выручки положительная, а где - отрицательная. Это очень важная для нас информация, нужно исправлять отрицательную динамику.
       
</div> 	


<br/>
<div class="alert alert-info">
Добавил визуализации прибыльности и выручки на график распределения сегментов и сделал выводы о динамике и прибыльности сегментов
</div>

Проанализируем частоту маркетинговых активностей для премиальных пользователей за последние 6 месяцев.

In [ ]:
def categorize_marketing_activity(activity_level):
    if activity_level <= 2:
        return "Низкая активность (0-2)"
    elif activity_level <= 4:
        return "Средняя активность (3-4)"
    elif activity_level <= 6:
        return "Высокая активность (5-6)"
    else:
        return "Очень высокая активность (7+)"

premium["marketing_category"] = premium["маркет_актив_6_мес"].apply(categorize_marketing_activity)
marketing_categories = premium["marketing_category"].value_counts().sort_index()


plt.figure(figsize = (12, 9))

colors = ["#E3F2FD", "#2196F3", "#0D47A1", "#001B5E"]

plt.pie(marketing_categories.values,
        labels = [f"{cat}\n({count} чел., {count/len(premium)*100:.1f}%)"
                for cat, count in marketing_categories.items()],
        colors = colors,
        autopct = "%1.1f%%",
        startangle = 90,
        explode = (0.05, 0.05, 0.05, 0.05),
        shadow = True,
        textprops = {"fontsize": 11, "fontweight": "bold"})

plt.title("Уровень маркетинговой активности в приоритетном сегменте\n(Высокая прибыль + Высокая необходимость удержания)",
          fontsize = 14, fontweight = "bold", pad = 20)

avg_activity = premium["маркет_актив_6_мес"].mean()
median_activity = premium["маркет_актив_6_мес"].median()
max_activity = premium["маркет_актив_6_мес"].max()

stats_text = f"Средняя активность: {avg_activity:.2f}\n" \
             f"Медиана: {median_activity:.1f}\n" \
             f"Максимум: {max_activity:.1f}\n" \
             f"Всего пользователей: {len(premium)}"

plt.text(-1.8, -1.3, stats_text,
         fontsize = 10, bbox = dict(boxstyle = "round,pad=0.5", facecolor = "lightblue", alpha = 0.8))

plt.axis("equal")
plt.tight_layout()
plt.show()

Для большинства пользователей из премиальной категории с высокой вероятноситью снижения активности за последние 6 месяцев наблюдалась высокая либо средняя маркетинговая активность. Вероятно, пользователям надоедает излишне назойливая реклама. Есть смысл подумать об изменениях маркетиинговой статегии для премиальных пользователей.

In [ ]:
col = "неоплаченные_продукты_штук_квартал"
x = pd.to_numeric(premium[col], errors="coerce").copy()
x = x.clip(lower = 0)

def categorize_unpurchased(n):
    if n == 0:
        return "0"
    elif n <= 2:
        return "1–2"
    elif n <= 5:
        return "3–5"
    elif n <= 10:
        return "6–10"
    else:
        return "11+"

premium["unpaid_category"] = x.apply(categorize_unpurchased)

order = ["0", "1–2", "3–5", "6–10", "11+"]
counts = premium["unpaid_category"].value_counts().reindex(order, fill_value=0)

avg = x.mean()
median = x.median()
p90 = x.quantile(0.90)
maxv = x.max()
n_total = len(x)
n_na = premium[col].isna().sum()

plt.figure(figsize = (12, 9))

colors = ["#E3F2FD", "#90CAF9", "#42A5F5", "#1E88E5", "#0D47A1"]
labels = [
    f"{cat}\n({cnt} чел., {cnt/len(premium)*100:.1f}%)"
    for cat, cnt in counts.items()
]

plt.pie(
    counts.values,
    labels = labels,
    colors = colors,
    autopct = "%1.1f%%",
    startangle = 90,
    explode = (0.05, 0.05, 0.05, 0.05, 0.05),
    shadow = True,
    textprops = {"fontsize": 11, "fontweight": "bold"}
)

plt.title(
    "Невыкупленные товары из корзины за квартал\n(распределение по числу позиций, шт.)",
    fontsize = 14, fontweight = "bold", pad = 20
)

stats_text = (
    f"Среднее: {avg:.2f}\n"
    f"Медиана: {median:.1f}\n"
    f"90-й перцентиль: {p90:.1f}\n"
    f"Максимум: {maxv:.0f}\n"
    f"Всего наблюдений: {n_total}\n"
    f"Пустых значений: {n_na}"
)

plt.text(
    -1.9, -1.35, stats_text,
    fontsize=10,
    bbox = dict(boxstyle = "round,pad=0.5", facecolor = "lightblue", alpha = 0.8)
)

plt.axis("equal")
plt.tight_layout()
plt.show()


У большинства премиальных пользователей много невыкупленных товаров в карзине за последние месяцы. Вероятно, стоит продумать методы напоминаний и поощрений покупки невыкупленных товаров.

## Итоговые выводы

### Задача

Была поставлена **задача** построения модели для **предсказания покупательской активности** клиентов и выяснения **способов их удержания**.

### Подготовка данных

Были получены и подготовлены 4 файла с данными:
- унификация называний признаков
- фильтрация дубликатов
- преобразования типов данных
- обработаны пропуски
- исправлены описки
- расспотрена описательная статистика
- построены графики частот для количественных и качественных признаков
- обхеденены наборы данных из 4 файлов

### Поиск модели

Для поиска модели были **опробованы 4 алгоритма** классификации **с разными наборами** целесообразных **гиперпараметров** случайным образом.

### Лучшая модель классификации

В качестве **лучшей модели предсказания активности** клиента была получена модель **ближайших соседей с точностью** ROC AUC близкой к **идеальной**.

### **Идеи удержания клиентов**

В качестве основного для бизнеса был выделен **"премиальный" сегмент клиентов** с относительно высокой прибылью для бизнеса (~ 19% от всех пользователей), **теряющих интерес** к сервису.

**Замечания для маркетологов**:

Динамика выручки:
- Стабильно высокая выручка от премиальных пользователей в сегментах кухонной посуды и косметики и аксессуаров
- За последние два месяца упала выручка в сегментах техники для красоты и здоровья, товаров для детей
- За последние 2 месяца повысилась выручка в сегментах мелкой бытовой техники и электроники, домашнего текстиля

Категории товаров в премиальном сегменте пользователей приносящие наибольшую прибыль компании:
1) Косметика и аксессуары
2) Товары для детей и дома
3) Техника для красоты и здоровья

**Рекомендации**:
- **Можно продумать специальное предложение подписки на сервис премиальным клиентам для повышения их удержания**, т.к. такой шаг будет актуален для большинства премиальных клиентов в силу отсутствия у них подписки.
- Предоставить **временную повышенную скидку** на все товары **для пользователей**, которые **покупают меньше 20% товаров со скидкой** с целью **возвращения их интереса к сервису**
- Возможно, стоит реже показывать премиальным пользователям маркетинговые сообщения т.к. они им надоедают. Стоит провести дополнительное исследование этой гипотезы.
- Стоит продумать методы напоминаний и поощрений покупки невыкупленных товаров.
- Детальнее продумать маркетинговую стратегию для "премиального" сегмента пользователей

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"></h2>
  
<b>На доработку🤔:</b>
    
Нужно будет провести анализ данных, выбрать сегмент, уточниться по анализу сегмента.
    
А зачем нам делать скидку тем клиентам, вероятность снижения активности которых и так низкая?
    
Второй пункт рекомендаций нужно развернуть - уточнить аргументацию. Какая проблема сейчас с премиальным сегментом? 
    
***
    
Например, если бы мы выбрали сегментом для анализа клиентов с высокой долей акционных покупок и высоким риском снижения активности, наш анализ мог бы выглядеть так:
    
Мы видим, что у нас есть группа клиентов, которая практически все покупала только по акции. Выше я описал, что еси бы доступ к акциям был равный, то и распределение доли акционных покупок было близкое к нормальному, но точно не бимодальным, как мы наблюдаем. Следовательно, у нас есть группа, которую привлекли специальными акциями на определенные товары.
    
Это может объяснять, почему «акционщики» просматривают сильно меньше страниц и проводят меньше времени на сайте − они приходят за конкретными товарами и не сильно заинтересованы в исследовании остального ассортимента.
    
Можно предположить, что в какой-то момент акции для этой группы закончились. Это объясняет, почему клиенты со сниженной активностью имеют много неоплаченных товаров в корзине (цена изменилась, покупка отложилась).
    
Тогда можно посмотреть: а какова выручка у таких клиентов? Выгодно ли нам продлевать акции. Видим, что еще как выгодно. Тогда логично будет уточнить, а какие категории товаров популярны у нашего сегмента.
    
Для этого мы можем посмотреть суммарную выручку с группировкой по категориям товаров и с разбивкой по периодам (будут три периода). Можно реализовать это с помощью сводных таблиц и визуализировать. 
    
Для каждого из популярных товаров мы увидим динамику для нашего сегмента. Увидим, какие товары были популярны и снизили популярность. Если это товары длительного потребления (для примера - фены или утюги), то вряд ли клиентам будет нужно снова покупать ровно эти товары, тогда логично предложить им акции на что-то смежное.
    
Дополнительно важно проанализировать потенциально влияющие на отток признаки, которые у нас есть:
    
* количество ошибок на сайте - различается ли оно у активных и снижающих активность клиентов? Есть ли какие-то аномалии именно для нашего сегмента?
    
* премиум подписка - влияет ли на активность, а должна ли влиять? Если нет, то возомжно это потенциально полезный инструмент удержания, который почему-то пока не работает. 
</div>




<br/>
<div class="alert alert-info">
Переписал выводы на основании более детального анализа премиального сегмента пользователей после использвания валидной модели оценки активности 
</div>

<div style="border:solid Chocolate 2px; padding: 40px">

**Общий вывод по проекту**
    
Спасибо за твой проект! Эта работа также была достаточно объемной и требовала концентрации внимания. По структуре проекта, логичности действий и общему подходу все замечательно.
    
**Отмечу положительные моменты проекта🚀**
    
* Хорошие решения в коде, есть пояснения и аргументации.
* Корректное обучение моделей, использование пайплайнов.

    
**На что стоит обратить внимание🤔**
    
* Важно уточниться по неактивным клиентам.
* Нужно поправить по объединению таблиц. В результирующей таблице должно быть 1296 строк. Все id в строках должны быть уникальными, важно избежать дублирований.
* Важно в исследовательском анализе рассмотреть отдельно подвыборки для клиентов со снижающейся и с постоянной активностью (делим по целевому признаку).
* Не стоит удалять признаки из периодов. Эти признаки нужны. При корреляции до 0.9 можно оставить оба признака.
* Нужно поправить по анализу shap.
* Необходимо определить сегмент для анализа по результатам модели - вероятностям классов.
* Рекомендую также углубить и исследование сегмента: там, где можем, смотрим на динамику в категориях товаров. Я бы посмотрел на данные в разрезе акции. Думаю, как минимум, это подтвердит наши выводы, ну и возможно как-то их уточнит.
* Эти и остальные замечания я также написал по ходу проекта.    

**Желаю удачи и жду твой проект на повторное ревью!**😉
    
    
</div>


<div style="border:solid Chocolate 2px; padding: 40px">

**Комментарий ко второй проверке**

Спасибо за качественную доработку. Практически все верно. Однако еще кое-что осталось.

Важно поправить по загрузке датасетов. Также оставл несколько «желтых» комментариев-рекомендаций.

**Желаю удачи и жду твой проект на повторное ревью!**😉
</div>


<div style="border:solid Chocolate 2px; padding: 40px">

**Комментарий к третьей проверке**

* Нужно поправить по выводу датасетов в начале проекта. 
* Важно реализовать корректный способ расчета важности признаков и визуализировать его на графике. Используй любой удобный и известные тебе способ.
    
**Желаю удачи и жду твой проект на повторное ревью!**😉
</div>
